#  Обратное распространение ошибки

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann
* http://cs231n.stanford.edu/handouts/linear-backprop.pdf
* https://www.adityaagrawal.net/blog/deep_learning/bprop_fc
* https://en.wikipedia.org/wiki/Stochastic_gradient_descent

## Задачи для совместного разбора

1\. Реализуйте обратное распространение ошибки для модели нейрона с квадратичной функцией потерь при условии, что на вход нейрону поступает вектор `inputs`. Проверьте корректность вычисления градиентов, воспользовавшись возможностями по автоматическому дифференцированию `torch`.

In [ ]:
class Neuron:
  def __init__(self, weights, bias):
    self.weights = weights
    self.bias = bias

  def forward(self, inputs):
    return inputs @ self.weights + self.bias

  def backward(self, inputs, dldy):
    self.dw = dldy * inputs
    self.db = dldy

In [ ]:
class Loss:
  def forward(self, y_pred, y_true):
    return (y_pred - y_true) ** 2

  def backward(self, y_pred, y_true):
    # dL/dy~
    self.dypred = 2*(y_pred - y_true)

In [ ]:
import torch as th

x = th.tensor([2.0, 3.0])
y = th.tensor(5.0)

neuron = Neuron(weights=th.tensor([3.5, 4.5]), bias=0.5)
criterion = Loss()

y_pred = neuron.forward(x)
loss = criterion.forward(y_pred, y)

criterion.backward(y_pred, y)
neuron.backward(x, criterion.dypred)

In [ ]:
neuron.weights -= lr* neuron.dw
neuron.bias -= lr* neuron.db

2\. Настройте модель нейрона, используя метод стохастического градиентного спуска

## Задачи для самостоятельного решения

In [2]:
import torch

<p class="task" id="1"></p>

### 1
Реализуйте обратное распространение ошибки для модели нейрона с функцией потерь MSE при условии, что на вход нейрону поступает пакет (двумерный тензор) `inputs`. Проверьте корректность вычисления градиентов, воспользовавшись возможностями по автоматическому дифференцированию `torch`.

$$\mathbf{X} = \begin{bmatrix}
x_{10} & x_{11} & \ldots & x_{1m} \\
x_{20} & x_{21} & \ldots & x_{2m} \\
\vdots & \vdots & \ddots & \vdots \\
x_{k0} & x_{k1} & \ldots & x_{km} \\
\end{bmatrix}
\mathbf{Y} = \begin{bmatrix}
y_{1} \\
y_{2} \\
\vdots \\
y_{k} \\
\end{bmatrix}
\mathbf{W} = \begin{bmatrix}
w_{0} \\
w_{1} \\
\vdots \\
w_{m} \\
\end{bmatrix}$$

$$\hat{\mathbf{Y}} = \mathbf{X}\times \mathbf{W}$$

$$L = \frac{1}{k}\sum_{k}{(\hat{y_k}-y_k)^2}$$

$$\nabla_{\hat{\mathbf{Y}}} L=\begin{bmatrix}
\frac{\partial L}{\partial \hat{y_1}} \\
\frac{\partial L}{\partial \hat{y_2}} \\
\vdots \\
\frac{\partial L}{\partial \hat{y_k}} \\
\end{bmatrix} = \frac{2}{k}\begin{bmatrix}
\hat{y_1} - y_1 \\
\hat{y_2} - y_2 \\
\vdots \\
\hat{y_k} - y_k \\
\end{bmatrix}$$

$$\boldsymbol{\nabla_{\mathbf{W}} L = \mathbf{X}^T\nabla_{\hat{\mathbf{Y}}} L}$$

In [43]:
class Neuron(object):
    def __init__(self, weights, bias):
        self.weights = weights
        self.bias = bias

    def forward(self, inputs):
        return (inputs @ self.weights) + self.bias

    def backward(self, inputs, dldy, lr):
        self.dw = inputs.T @ dldy
        self.db = dldy

        self.lr = lr

        self.weights -= self.lr * self.dw
        self.bias -= self.lr * self.db

class MSELoss:
    def forward(self, y_pred, y_true):
        return torch.mean((y_pred - y_true) ** 2)

    def backward(self, y_pred, y_true):
        #dL/dy~
        self.dypred = 2/y_pred.shape[0]*(y_pred - y_true)

In [44]:
x = torch.randn(3, 4)
weights = torch.randn(4, 1)

x_clone = x.clone()
weights_clone = weights.clone().requires_grad_(True)

y_true = torch.tensor([[0.0], [1.0], [0.0]])

neuron = Neuron(weights, 0.5)
criterion = MSELoss()

y_pred = neuron.forward(x)
loss = criterion.forward(y_pred, y_true)

criterion.backward(y_pred, y_true)
neuron.backward(x, criterion.dypred, 0.03)

print(neuron.dw)

tensor([[ 0.1062],
        [ 0.3719],
        [-0.2518],
        [-0.1251]])


In [45]:
y_pred = x_clone @ weights_clone + 0.5

loss = torch.mean((y_pred - y_true)**2)

loss.backward()
weights_clone.grad

tensor([[ 0.1062],
        [ 0.3719],
        [-0.2518],
        [-0.1251]])

<p class="task" id="2"></p>

### 2
Настройте модель нейрона, используя метод мини-пакетного градиентного спуска. Используйте обратное распространение ошибки, реализованное самостоятельно.

In [37]:
from sklearn.datasets import make_regression
import torch as th
torch.manual_seed(42)

In [89]:
from sklearn.datasets import make_regression
import torch as th

X, y, coef = make_regression(n_features=4, n_informative=4, coef=True, bias=0.5, random_state=42)
X = th.FloatTensor(X)
y = th.FloatTensor(y).reshape(-1, 1)

In [90]:
X.shape, y.shape

(torch.Size([100, 4]), torch.Size([100, 1]))

In [32]:
class Neuron(object):
    def __init__(self, weights, bias):
        self.weights = weights
        self.bias = bias

    def forward(self, inputs):
        return (inputs @ self.weights) + self.bias

    def backward(self, inputs, dldy, lr):
        self.dw = inputs.T @ dldy
        self.db = dldy

        self.lr = lr

        self.weights -= self.lr * self.dw
        self.bias -= self.lr * self.db

class MSELoss:
    def forward(self, y_pred, y_true):
        return ((y_pred - y_true) ** 2).mean(dim=1)

    def backward(self, y_pred, y_true):
        #dL/dy~
        self.dypred = 2/y_pred.shape[0]*(y_pred - y_true)

In [33]:
neuron = Neuron(weights = torch.randn(X.shape[1], 1), bias = 0.5)
criterion = MSELoss()

for epoch in range(100):
    for i in range(0, X.shape[0], 25):
        batch_X = X[i:i+25]
        batch_y = y[i:i+25]

        y_pred = neuron.forward(batch_X)
        loss = criterion.forward(y_pred, batch_y)

        criterion.backward(y_pred, batch_y)
        neuron.backward(batch_X, criterion.dypred, 0.03)

In [34]:
neuron.weights

tensor([[ 5.6728],
        [86.0448],
        [27.1529],
        [41.2401]])

In [35]:
coef

array([ 5.63754967, 86.47223763, 27.34070719, 41.48195023])

<p class="task" id="3"></p>

### 3
Реализуйте обратное распространение ошибки для модели полносвязного слоя с функцией потерь MSE при условии, что на вход нейрону поступает пакет (двумерный тензор) `inputs`.  Проверьте корректность вычисления градиентов, воспользовавшись возможностями по автоматическому дифференцированию `torch`.

$$\mathbf{X} = \begin{bmatrix}
x_{10} & x_{11} & \ldots & x_{1m} \\
x_{20} & x_{21} & \ldots & x_{2m} \\
\vdots & \vdots & \ddots & \vdots \\
x_{k0} & x_{k1} & \ldots & x_{km} \\
\end{bmatrix}
\mathbf{Y} = \begin{bmatrix}
y_{1} \\
y_{2} \\
\vdots \\
y_{k} \\
\end{bmatrix}
\mathbf{W} = \begin{bmatrix}
w_{01} & w_{02} & \ldots & w_{0n} \\
w_{11} & w_{12} & \ldots & w_{1n} \\
\vdots & \vdots & \ddots & \vdots \\
w_{m1} & w_{m2} & \ldots & w_{mn} \\
\end{bmatrix}$$

$$\hat{\mathbf{Y}} = \mathbf{X}\times \mathbf{W}$$

$$\nabla_{\hat{\mathbf{Y}}} L = \begin{bmatrix}
\frac{\partial L}{\partial \hat{y_{11}}} & \ldots & \frac{\partial L}{\partial \hat{y_{1n}}} \\
\vdots & \vdots & \vdots \\
\frac{\partial L}{\partial \hat{y_{k1}}} & \ldots & \frac{\partial L}{\partial \hat{y_{kn}}} \\
\end{bmatrix}$$

$$\boldsymbol{\nabla_{\mathbf{W}} L = \mathbf{X}^T\times \nabla_{\hat{\mathbf{Y}}} L}$$
$$\boldsymbol{\nabla_{\mathbf{X}} L = \nabla_{\hat{\mathbf{Y}}} L\times \mathbf{W}^T}$$

In [36]:
class Neuron(object):
    def __init__(self, weights, bias):
        self.weights = weights
        self.bias = bias

    def forward(self, inputs):
        return (inputs @ self.weights) + self.bias

    def backward(self, inputs, dldy, lr):
        self.dw = inputs.T @ dldy
        self.dx = dldy @ self.weights.T
        self.db = dldy

        self.lr = lr

        self.weights -= self.lr * self.dw
        self.bias -= self.lr * self.db

class MSELoss:
    def forward(self, y_pred, y_true):
        return torch.mean((y_pred - y_true) ** 2)

    def backward(self, y_pred, y_true):
        #dL/dy~
        self.dypred = 2/(y_pred.shape[0]*y_pred.shape[1])*(y_pred - y_true)

In [41]:
torch.manual_seed(42)

x = torch.randn(3, 2)
weights = torch.randn(2, 4)

y = torch.tensor([[0.0], [1.0], [0.0]])

neuron = Neuron(weights, 0.5)
criterion = MSELoss()

y_pred = neuron.forward(x)
loss = criterion.forward(y_pred, y)

criterion.backward(y_pred, y)
neuron.backward(x, criterion.dypred, 0.03)

print('Градиент по weights:', neuron.dw)
print('Градиент по X:', neuron.dx)

Градиент по weights: tensor([[ 0.4683, -0.1957,  0.0816, -0.1077],
        [ 0.0982, -0.0425,  0.0189, -0.0397]])
Градиент по X: tensor([[ 0.5196,  0.2124],
        [ 0.0538,  0.1577],
        [-0.8729, -0.2282]])


In [42]:
torch.manual_seed(42)

x = torch.randn(3, 2, requires_grad = True)
weights = torch.randn(2, 4, requires_grad = True)

y = torch.tensor([[0.0], [1.0], [0.0]])

y_pred = x @ weights + 0.5

loss = torch.mean((y_pred - y)**2)

loss.backward()
print('Градиент по weights:', weights.grad)
print('Градиент по X:', x.grad)

Градиент по weights: tensor([[ 0.4683, -0.1957,  0.0816, -0.1077],
        [ 0.0982, -0.0425,  0.0189, -0.0397]])
Градиент по X: tensor([[ 0.5196,  0.2124],
        [ 0.0538,  0.1577],
        [-0.8729, -0.2282]])


<p class="task" id="4"></p>

### 4
Настройте полносвязный слой, используя метод пакетного градиентного спуска. Используйте обратное распространение ошибки, реализованное самостоятельно.

In [60]:
import plotly.graph_objects as go

In [47]:
from sklearn.datasets import make_regression
import torch as th

X, y, coef = make_regression(n_features=4, n_informative=4, coef=True, bias=0.5, random_state=42)
X = th.FloatTensor(X)
y = th.FloatTensor(y).reshape(-1, 1)

<p class="task" id="5"></p>

### 5
Используя решения предыдущих задач, создайте нейросеть и решите задачу регрессии.

Предлагаемая архитектура:
1. Полносвязный слой с 10 нейронами
2. Полносвязный слой с 1 нейроном

В процессе обучения сохраняйте прогнозы промежуточных моделей. Визуализируйте облако точек и промежуточные прогнозы не менее 4 промежуточных моделей. Визуализируйте прогнозы итоговой модели.

In [48]:
class Neuron(object):
    def __init__(self, weights, bias):
        self.weights = weights
        self.bias = bias

    def forward(self, inputs):
        return (inputs @ self.weights) + self.bias

    def backward(self, inputs, dldy, lr):
        self.dw = inputs.T @ dldy
        self.dx = dldy @ self.weights.T
        self.db = dldy

        self.lr = lr

        self.weights -= self.lr * self.dw
        self.bias -= self.lr * self.db

        return self.dx

class MSELoss:
    def forward(self, y_pred, y_true):
        return torch.mean((y_pred - y_true) ** 2)

    def backward(self, y_pred, y_true):
        #dL/dy~
        self.dypred = 2/(y_pred.shape[0]*y_pred.shape[1])*(y_pred - y_true)

In [49]:
class NeuralNetwork:
    def __init__(self):
        self.layer1 = Neuron(torch.randn(1, 10), 0.5)
        self.layer2 = Neuron(torch.randn(10, 1), 0.5)

    def forward(self, inputs):
        x = self.layer1.forward(inputs)
        y_pred = self.layer2.forward(x)
        return y_pred

    def backward(self, inputs, dldy, lr):
        dldx = self.layer2.backward(inputs, dldy, lr)
        self.layer1.backward(inputs, dldx, lr)

In [77]:
th.manual_seed(42)

X = th.linspace(-1, 1, 100).view(-1, 1)
y = X.pow(2) + 0.2 * th.rand(X.size())

In [76]:
criterion = MSELoss()
nn = NeuralNetwork()
preds = []

for epoch in range(1, 10001):
    y_pred = nn.forward(X)
    loss = criterion.forward(y_pred, y)

    criterion.backward(y_pred, y)
    nn.backward(X, criterion.dypred, 0.03)

    if epoch in [200, 300, 400, 500, 600]:
        preds.append(y_pred)

In [81]:
fig = go.Figure(
    [
        go.Scatter(name = 'epoch 200', x = X.ravel(), y = preds[0].ravel(), mode = 'markers'),
        go.Scatter(name = 'epoch 300', x = X.ravel(), y = preds[1].ravel(), mode = 'markers'),
        go.Scatter(name = 'epoch 400', x = X.ravel(), y = preds[2].ravel(), mode = 'markers'),
        go.Scatter(name = 'epoch 500', x = X.ravel(), y = preds[3].ravel(), mode = 'markers'),
        go.Scatter(name = 'epoch 600', x = X.ravel(), y = preds[4].ravel(), mode = 'markers'),
        go.Scatter(name = 'true values', x = X.ravel(), y = y.ravel(), mode = 'markers'),
    ]
)

fig.show()

<p class="task" id="6"></p>

### 6
Cоздайте нейросеть и решите задачу регрессии из предыдущей задачи.

Предлагаемая архитектура:
1. Полносвязный слой с 10 нейронами
2. Активация ReLU
3. Полносвязный слой с 1 нейроном

В процессе обучения сохраняйте прогнозы промежуточных моделей. Визуализируйте облако точек и промежуточные прогнозы не менее 4 промежуточных моделей. Визуализируйте прогнозы итоговой модели.

In [82]:
class Neuron(object):
    def __init__(self, weights, bias):
        self.weights = weights
        self.bias = bias

    def forward(self, inputs):
        return (inputs @ self.weights) + self.bias

    def backward(self, inputs, dldy, lr):
        self.dw = inputs.T @ dldy
        self.dx = dldy @ self.weights.T
        self.db = dldy

        self.lr = lr

        self.weights -= self.lr * self.dw
        self.bias -= self.lr * self.db

        return self.dx

class MSELoss:
    def forward(self, y_pred, y_true):
        return torch.mean((y_pred - y_true) ** 2)

    def backward(self, y_pred, y_true):
        #dL/dy~
        self.dypred = 2/(y_pred.shape[0]*y_pred.shape[1])*(y_pred - y_true)

In [83]:
class NeuralNetwork:
    def __init__(self):
        self.layer1 = Neuron(torch.randn(1, 10), 0.5)
        self.layer2 = Neuron(torch.randn(10, 1), 0.5)

    def forward(self, inputs):
        x = self.layer1.forward(inputs)
        x = torch.clip(x, min = 0) # relu
        y_pred = self.layer2.forward(x)
        return y_pred

    def backward(self, inputs, dldy, lr):
        dldx = self.layer2.backward(inputs, dldy, lr)
        dldx = dldx * (dldx > 0) #relu' = {0: x<0, 1:x>0}
        self.layer1.backward(inputs, dldx, lr)

In [84]:
th.manual_seed(42)

X = th.linspace(-1, 1, 100).view(-1, 1)
y = X.pow(2) + 0.2 * th.rand(X.size())

In [87]:
criterion = MSELoss()
nn = NeuralNetwork()
preds = []

for epoch in range(1, 10001):
    y_pred = nn.forward(X)
    loss = criterion.forward(y_pred, y)

    criterion.backward(y_pred, y)
    nn.backward(X, criterion.dypred, 0.03)

    if epoch in [400, 500, 600, 800, 1000]:
        preds.append(y_pred)

In [88]:
fig = go.Figure(
    [
        go.Scatter(name = 'epoch 400', x = X.ravel(), y = preds[0].ravel(), mode = 'markers'),
        go.Scatter(name = 'epoch 500', x = X.ravel(), y = preds[1].ravel(), mode = 'markers'),
        go.Scatter(name = 'epoch 600', x = X.ravel(), y = preds[2].ravel(), mode = 'markers'),
        go.Scatter(name = 'epoch 800', x = X.ravel(), y = preds[3].ravel(), mode = 'markers'),
        go.Scatter(name = 'epoch 1000', x = X.ravel(), y = preds[4].ravel(), mode = 'markers'),
        go.Scatter(name = 'true values', x = X.ravel(), y = y.ravel(), mode = 'markers'),
    ]
)

fig.show()

## Обратная связь
- [ ] Хочу получить обратную связь по решению